# Spin Qubit Pulse Sequences with the HDAWG

This notebook shows you how to use pulse sequences with an HDAWG to realize various experiments. While the sequences are mostly general, they are adapted to those typically used for spin qubits by adding the gate pulsing to control between Coulomb and spin blockade.


* Make use of the calibration to adjust the parameters
* Randomized benchmarking with IQ tomography and Bayesian update, which will be used as feedback.
* QND measurements. with repetition of sections

# General Imports

In [1]:
%config IPCompleter.greedy=True

# import matplotlib.pyplot as plt
import numpy as np
import time
import zhinst.core
import laboneq
from laboneq.simple import *
import helpers

# install_token("LabOneQ.AllQuantumInOne.")
DO_EMULATION = False # run in emulation mode by default

print(laboneq.__version__)

2.7.0


# Device Setup

Device specific parameters are still required.

## Create device setup

In [2]:
descriptor="""
instrument_list:
  HDAWG:
  - address: DEV8721
    uid: device_hdawg
    interface: usb
connections:
  device_hdawg:
    - rf_signal: q0/fg2_line
      ports: [SIGOUTS/0]
    - rf_signal: q0/fg4_line
      ports: [SIGOUTS/1]
    - iq_signal: q0/drive_line
      ports: [SIGOUTS/2, SIGOUTS/3]
"""

device_setup = DeviceSetup.from_descriptor(
    descriptor,
    server_host="127.0.0.1",
    server_port="8004",
    setup_name="ZI_HDAWG",
)

c:\Users\Dobby\Anaconda3\lib\site-packages\laboneq\dsl\device\_device_setup_generator.py:1104: FutureWarning: 'instrument_list' section is deprecated in setup descriptor, use 'instruments' instead.
  warnings.warn(


# S-T readout

## Define parameters for experiment

In [9]:
points_map={
    'initial_point': [0.1, 0.7],
    'anchor_point': [-0.2, 0.2],
    'readout_point': [0.3, 0.2], 
    'control_point': [0.4, 0.9],}

points_map['unload_point'] = points_map['initial_point']

times_map = {'initial_time': 10e-6, 
             'anchor_time': 4e-6, 
             'readout_time': 6e-6, 
             'control_time': 8e-6}

times_map['unload_time']=times_map['initial_time']

In [10]:
## constant definition
LEN_COULOMB_CYCLE = 200e-6
X90_DURATION = 2e-6

START = 1e-6
STOP = 7e-6
STEPS = 5
NUM_REP = 3

## define length sweep parameter
length_sweep_parameter = LinearSweepParameter(
    uid="length_sweep", start=START, stop=STOP, count=STEPS,
)

## Experiment pulses
Connections: A- 2wave, B- 3 wave, C - 2 Trig

Question:
9. The amplitude is related to the level amplitude in defining the pulse
10. Channel 3 and 4 seems to have a 6ms delay comparing to channel 1 and 2
11. Pulse viewer doesn't exist in my installation
12. Not all pulses are properly compressed.

Solved:
6. Trigger is not outputing properly. - (Issue coming from channel grouping. How can I control the second group? Or does labone q control the second group?)
7. It doesn't seems that the the driver take the AWG sampling rate info etc - One can not change the sampling rate as this is defined by laboneq.

Requirement: when section time doesn't equal to gate1 and gate2, I dont want it to pad zeros. Basically meaning that section time should always be equal to pulse time. How do I hold a voltage between pulses?

In [11]:
# define pulse shapes
drive_pulse = pulse_library.const(
    uid="rabi_drive_pulse", length=400e-6, amplitude=1
)

@pulse_library.register_pulse_functional
def ramp(x,start=0,stop=1 , **_):
    pulse=start+ (stop-start)*(x+1)/2
    return pulse

@pulse_library.register_pulse_functional
def complexIQ(x,cycles=5, **_):
    pulse=np.cos(np.pi*cycles*x)+ 1j*np.sin(np.pi*cycles*x)
    return pulse

compress_level_pulse=pulse_library.const(uid="compress_level",length=times_map['initial_time'],amplitude=1,can_compress=True)
level_pulse = pulse_library.const(uid="level",length=times_map['control_time'],amplitude=1)
ramp_pulse = ramp(uid="ramp",start=0.2,stop=0.7,length=400e-9, amplitude=1)
complex_IQ=complexIQ(uid="complex",freq=5,cycles=200e-6, amplitude=1)

In [12]:
## Create Experiment
exp = Experiment(
    "Singlet-triplet qubit example",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("gate1"),
        ExperimentSignal("gate2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid=("shots"), count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL
):
    with exp.sweep(uid='sweep_rabi',parameter=length_sweep_parameter):
        with exp.section(
            uid=("unload"),
            length=times_map['unload_time'], 
            alignment=SectionAlignment.RIGHT,
        ):
            # exp.play(signal="gate1", pulse=level_pulse, amplitude=points_map['unload_point'][0],length=times_map['unload_time']) 
            # exp.play(signal="gate2", pulse=level_pulse, amplitude=points_map['unload_point'][1],length=times_map['unload_time'])
            exp.play(signal="gate1", pulse=compress_level_pulse, amplitude=points_map['unload_point'][0]) 
            exp.play(signal="gate2", pulse=compress_level_pulse, amplitude=points_map['unload_point'][1])
        with exp.section(
            uid=("anchor"),
            length=times_map['anchor_time'],
            alignment=SectionAlignment.RIGHT,
        ):
            exp.play(signal="gate1", pulse=level_pulse, amplitude=points_map['anchor_point'][0],length=times_map['anchor_time']) 
            exp.play(signal="gate2", pulse=ramp_pulse, amplitude=1,length=times_map['anchor_time'])
        with exp.section(
            uid=("control"),
            length=times_map['control_time'],
            alignment=SectionAlignment.RIGHT,
        ):
            exp.play(signal="gate1", pulse=level_pulse, amplitude=points_map['control_point'][0],length=times_map['control_time'])
            exp.play(signal="gate2", pulse=level_pulse, amplitude=points_map['control_point'][1],length=times_map['control_time'])
            exp.play(signal="drive", pulse=complex_IQ, length=length_sweep_parameter)
        with exp.section(uid="readout", 
            length=times_map['readout_time'],
            trigger={"gate1": {"state": 1}}, # Play on the maker output(state:1) on the 'drive' line
            alignment=SectionAlignment.RIGHT):
            exp.play(signal="gate1", pulse=level_pulse, amplitude=points_map['readout_point'][0],length=times_map['readout_time'])
            exp.play(signal="gate2", pulse=ramp_pulse, amplitude=-1,length=times_map['readout_time'])
        with exp.section(
            uid="outer_trigger",
            length=times_map['readout_time'],
            trigger={"drive": {"state": 1}},
            alignment=SectionAlignment.RIGHT,
        ):

            exp.reserve(signal="drive")
            # exp.play(signal="drive", pulse=drive_pulse, amplitude=0.01,length=times_map['readout_time']/2,marker = {"marker1": {"enable": True}},)
            # exp.play(signal="drive", pulse=drive_pulse, amplitude=0.01,length=times_map['readout_time']/2,)
        

In [13]:
# shortcut to the logical signal group q0
lsg = device_setup.logical_signal_groups["q0"].logical_signals

# define signal map
map_signals = {
    "drive" : lsg["drive_line"],
    "gate1" : lsg["fg2_line"],
    "gate2" : lsg["fg4_line"]
}

In [14]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)
# set experiment calibration and signal map
exp.set_signal_map(map_signals)

if not session.connection_state.emulated:
    instrument_serial = device_setup.instrument_by_uid("device_hdawg").address
    device = session.devices[instrument_serial]
    device.triggers.out[2].delay(23.9e-9)

session.run(exp)

2023.06.05 12:55:31.999 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is c:\Users\Dobby\Desktop\HDAWG\laboneq_output\log
2023.06.05 12:55:32.001 laboneq.controller.controller  INFO   VERSION: laboneq 2.7.0
2023.06.05 12:55:32.002 laboneq.controller.devices.device_collection INFO   Connecting to data server at 127.0.0.1:8004
2023.06.05 12:55:32.082 laboneq.controller.communication INFO   Connected to Zurich Instruments LabOne Data Server version 23.02 at 127.0.0.1:8004
2023.06.05 12:55:32.313 laboneq.controller.devices.device_collection INFO   Configuring the device setup
2023.06.05 12:55:32.494 laboneq.controller.devices.device_collection INFO   The device setup is configured
2023.06.05 12:55:32.855 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.06.05 12:55:32.868 laboneq.compiler.scheduler.scheduler INFO   Schedule completed
2023.06.05 12:55:32.938 laboneq.compil

Results(experiment=Experiment(uid='Singlet-triplet qubit example', signals={'drive': ExperimentSignal(uid='drive', calibration=None, mapped_logical_signal_path='/logical_signal_groups/q0/drive_line'), 'gate1': ExperimentSignal(uid='gate1', calibration=None, mapped_logical_signal_path='/logical_signal_groups/q0/fg2_line'), 'gate2': ExperimentSignal(uid='gate2', calibration=None, mapped_logical_signal_path='/logical_signal_groups/q0/fg4_line')}, version=DSLVersion.V3_0_0, epsilon=0.0, sections=[AcquireLoopRt(uid='shots', alignment=SectionAlignment.LEFT, execution_type=ExecutionType.REAL_TIME, length=None, play_after=None, children=[Sweep(uid='sweep_rabi', alignment=SectionAlignment.LEFT, execution_type=ExecutionType.REAL_TIME, length=None, play_after=None, children=[Section(uid='unload', alignment=SectionAlignment.RIGHT, execution_type=ExecutionType.REAL_TIME, length=1e-05, play_after=None, children=[PlayPulse(signal='gate1', pulse=PulseFunctional(function='const', uid='compress_level', 

### View experiment in pulse sheet viewer

To-do: 
1. Add stability diagram visualization

In [15]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet(".\laboneq_output\ST pulse test", compiled_exp)

2023.06.05 12:55:35.446 laboneq.pulse_sheet_viewer.pulse_sheet_viewer INFO   Writing html file to c:\Users\Dobby\Desktop\HDAWG\laboneq_output\ST pulse test_2023-06-05-12-55-35.html


c:\Users\Dobby\Desktop\HDAWG\laboneq_output\ST pulse test_2023-06-05-12-55-35.html

In [16]:
helpers.interactive_psv(compiled_exp)

# 2D sweep example, wait time vs phase of second pulse

## Experiment

In [ ]:
## constant definition
X90_DURATION = 10e-9  # [s]

START = 0
STOP = 2 * np.pi
STEPS = 5
NUM_REP = 2

START_DELAY = 0
STOP_DELAY = LEN_COULOMB_CYCLE / 2 - 2 * X90_DURATION
STEPS_DELAY = 3


In [ ]:
## define phase sweep parameter
sweep_phase = LinearSweepParameter(
    uid="phase_sweep", start=START, stop=STOP, count=STEPS
)
sweep_delay = LinearSweepParameter(
    uid="Ramsey_delay", start=START_DELAY, stop=STOP_DELAY, count=STEPS_DELAY
)

print(sweep_phase.values / np.pi)

In [ ]:
## Create Experiment
exp = Experiment(
    "Ramsey variant I",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("gate1"),
        ExperimentSignal("gate2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid="shots", count=NUM_REP, averaging_mode=AveragingMode.CYCLIC
):
    with exp.sweep(uid="sweep_delay", parameter=sweep_delay):
        with exp.sweep(uid="sweep_phase", parameter=sweep_phase):
            with exp.section(
                uid="qubit_excitation",
                alignment=SectionAlignment.RIGHT,
                length=LEN_COULOMB_CYCLE,
            ):
                exp.play(signal="gate1", pulse=coulomb_pulse, amplitude=0.5)
                exp.play(signal="gate1", pulse=coulomb_pulse, amplitude=0.75)

                exp.play(signal="gate2", pulse=coulomb_pulse, amplitude=0.5)
                exp.play(signal="gate2", pulse=coulomb_pulse, amplitude=0.75)

                exp.play(signal="drive", pulse=drive_pulse, set_oscillator_phase=0)
                exp.delay(signal="drive", time=sweep_delay)
                exp.play(
                    signal="drive",
                    pulse=drive_pulse,
                    increment_oscillator_phase=sweep_phase,
                )
            with exp.section(
                uid="qubit_readout",
                alignment=SectionAlignment.RIGHT,
                length=LEN_READOUT,
            ):
                exp.play(signal="gate1", pulse=coulomb_readout, amplitude=0.3)
                exp.play(signal="gate2", pulse=coulomb_readout, amplitude=0.3)
            with exp.section(
                uid="outer_trigger",
                length=LEN_READOUT,
                trigger={"drive": {"state": 1}},
                alignment=SectionAlignment.RIGHT,
            ):
                with exp.section(
                    uid="inner_trigger",
                    length=LEN_READOUT - 100e-9,
                    trigger={"drive": {"state": 2}},
                ):
                    exp.reserve(signal="drive")


In [ ]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

session.run(exp)

## View experiment in pulse sheet viewer

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("2D example", compiled_exp)

